In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:14:08tcmalloc: large alloc 1147494400 bytes == 0x563d54c26000 @  0x7f90950a5615 0x563d1a95f3bc 0x563d1aa4018a 0x563d1a9621cd 0x563d1aa54b3d 0x563d1a9d6458 0x563d1a9d102f 0x563d1a963aba 0x563d1a9d62c0 0x563d1a9d102f 0x563d1a963aba 0x563d1a9d2cd4 0x563d1aa55986 0x563d1a9d2350 0x563d1aa55986 0x563d1a9d2350 0x563d1aa55986 0x563d1a9d2350 0x563d1a963f19 0x563d1a9a7a79 0x563d1a962b32 0x563d1a9d61dd 0x563d1a9d102f 0x563d1a963aba 0x563d1a9d2cd4 0x563d1a9d102f 0x563d1a963aba 0x563d1a9d1eae 0x563d1a9639da 0x563d1a9d2108 0x563d1a9d102f
     |█████████████████               | 1055.7 MB 1.2 MB/s eta 0:12:32tcmalloc: large alloc 1434370048 bytes == 0x563d9927c000 @  0x7f90950a5615 0x563d1a95f3bc 0x563d1aa4018a 0x563d1a9621cd 0x563d1aa54b3d 0x563d1a9d6458 0x563d1a9d102f 0x563d1a963aba 0x563d1a9d62c0 0x563d1a9d102f 0x563d1a963aba 0x563d1a9d2cd4 0x563d1aa55986 0x563d1a9d23

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 3.8 MB 23.3 MB/s 
     |████████████████████████████████| 6.5 MB 41.1 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
     |████████████████████████████████| 895 kB 34.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [12]:
r = requests.get('https://www.yelp.com/biz/biltmore-hotel-miami-coral-gables-coral-gables?osq=Hotels')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [13]:
reviews

["My husband and I stayed here a few days ago on our wedding night, absolutely gorgeous hotel! We stayed in the tower suite and it was gorgeous, it was very pricey but worth it as the property is beautiful and there's so much to do! The only downside to this beautiful hotel is the service. The service is terrible is this hotel, normally I don't notice but for the price per night, the service should be 5 stars. The elevator broke on the top floor, I was staying on floor 12 and had to take the stairs down to floor 5 just to get to another working elevator, also it took 2 hours for the cleaning service to bring up 1 mini bottle of shampoo and conditioner. I would definitely recommend staying at this hotel!! Especially if you enjoy swimming because they have the largest pool I've ever seen",
 'Dear Jackie, thank you for spending your wedding night with us and for sharing your feedback. We value you patronage and apologize for the lack of service and the elevator malfunction. We look forwar

In [14]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.array(reviews), columns=['review'])

In [16]:
df['review'].iloc[0]

"My husband and I stayed here a few days ago on our wedding night, absolutely gorgeous hotel! We stayed in the tower suite and it was gorgeous, it was very pricey but worth it as the property is beautiful and there's so much to do! The only downside to this beautiful hotel is the service. The service is terrible is this hotel, normally I don't notice but for the price per night, the service should be 5 stars. The elevator broke on the top floor, I was staying on floor 12 and had to take the stairs down to floor 5 just to get to another working elevator, also it took 2 hours for the cleaning service to bring up 1 mini bottle of shampoo and conditioner. I would definitely recommend staying at this hotel!! Especially if you enjoy swimming because they have the largest pool I've ever seen"

In [17]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [18]:
sentiment_score(df['review'].iloc[1])

5

In [19]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [20]:
df

,review,sentiment
0,My husband and I stayed here a few days ago on...,4
1,"Dear Jackie, thank you for spending your weddi...",5
2,Hotel rooms: I discovered bodily fluids and la...,4
3,"This past week, we stayed at the Biltmore. It ...",5
4,"Dear Cara, thank you for staying with us and f...",5
5,The beautiful Biltmore hotel in the heart of C...,5
6,"Dear Isabella, thank you for choosing the Bilt...",5
7,What a beautifully classic hotel! Our room wa...,5
8,"Dear Kate, thank you for staying and celebrati...",5
9,"Great hotel, obviously an icon in Miami. Howev...",3
